In [1]:
import os

import scanpy as sc

from scgen_code.scgen._scgen import SCGEN

Global seed set to 0
c:\Users\PavloBilinskyi\OneDrive - Ladoburn Europe Ltd\Documents\Documents (onedrive)\Education\master\Spring\Курсова\venv\lib\site-packages\pytorch_lightning\utilities\warnings.py:54: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  "pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6"
c:\Users\PavloBilinskyi\OneDrive - Ladoburn Europe Ltd\Documents\Documents (onedrive)\Education\master\Spring\Курсова\venv\lib\site-packages\pytorch_lightning\utilities\warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
import os

import scanpy as sc

from scgen_code.scgen._scgen import SCGEN

# Read the dataset
dataset = sc.read(os.path.join('data','train_kang.h5ad'),
                backup_url='https://drive.google.com/uc?id=1r87vhoLLq6PXAYdmyyd89zG90eJOFYLk')


cell_type = 'CD4T'
cell_types = ['NK', 'Dendritic', 'CD4T', 'B', 'FCGR3A+Mono', 'CD14+Mono', 'CD8T']

for i, cell_type in enumerate(cell_types):
    print(f'Leave-one-out experiment, leaving out cell type {cell_type}, step {i+1}/{len(cell_types)}')
    
    # prepare train and evaluation datasets
    mask_is_cell_type = dataset.obs["cell_type"] == cell_type
    train_adata = dataset[~(mask_is_cell_type & (dataset.obs["condition"] == "stimulated"))]
    train_adata = train_adata.copy()
    eval_adata_control = dataset[mask_is_cell_type & (dataset.obs["condition"] == "control")]
    eval_adata_stimulated = dataset[mask_is_cell_type & (dataset.obs["condition"] == "stimulated")]

    # setup data
    SCGEN.setup_anndata(train_adata, batch_key="condition", labels_key="cell_type")

    # create and save initial model
    model = SCGEN(train_adata)

    # train model
    model.train(
        max_epochs= 100,
        batch_size=32,
        early_stopping=False,
        early_stopping_patience=25
    )

    # save trained model
    save_dir = f"saved_models/leave_one_out/leave_out_{cell_type}"
    model.save(save_dir, overwrite=True)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Leave-one-out experiment, leaving out cell type NK, step 1/7
Epoch 2/100:   1%|          | 1/100 [01:07<1:51:40, 67.69s/it, loss=96.7, v_num=1]